In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cpu


In [4]:
with open('wizard_of_oz.txt') as file:
    text = file.read()

In [5]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
vocab_size = len(chars)

In [7]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)

In [8]:
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [9]:
## creating Batches
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batches(split):
    data = train_data if split== 'train'else val_data
    ix = torch.randint(len(data) - block_size,(block_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y= x.to(device),y.to(device)
    return x,y

x,y = get_batches('train')
print('inputs')
print(x)
print('targets')
print(y)
    
    

inputs
tensor([[72, 62, 60, 67,  1, 66, 78,  1],
        [58,  1, 56, 68, 75, 58, 71, 58],
        [58, 66, 23,  1, 55, 74, 73,  0],
        [ 1, 72, 54, 62, 57, 22,  0,  0],
        [58, 72,  1, 54, 57, 75, 54, 67],
        [54,  1, 68, 67, 56, 58,  1, 54],
        [68, 74,  1, 66, 54, 78,  1, 54],
        [54, 72,  1, 67, 68, 76,  1, 59]])
targets
tensor([[62, 60, 67,  1, 66, 78,  1, 68],
        [ 1, 56, 68, 75, 58, 71, 58, 57],
        [66, 23,  1, 55, 74, 73,  0, 73],
        [72, 54, 62, 57, 22,  0,  0,  3],
        [72,  1, 54, 57, 75, 54, 67, 56],
        [ 1, 68, 67, 56, 58,  1, 54,  1],
        [74,  1, 66, 54, 78,  1, 54, 72],
        [72,  1, 67, 68, 76,  1, 59, 58]])


In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,index,targets = None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # Batch_size, Time (text sequences), Channels 
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits, loss

    def generate(self,index,max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            logits, loss = self.forward(index)

            logits = logits[:,-1,:]

            probs = F.softmax(logits,dim=-1)

            index_next = torch.multinomial(probs,num_samples=1)

            index = torch.cat((index,index_next), dim = 1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context,
                                    max_new_tokens=500)[0].tolist())

print(generated_chars)

    


2&B]Jhw[PK]T?xv6WDT8XSfiPoEhmW'J"z]lOgHioY_OTJCUZM;6p8aR:[['6j;]"I:)PV8nz;MqX6EQFgr'Gq7F;61Z:tD3iRICKnlB?DniTXE*C vnSU)3Y
2d!pWg57laiIm1Xntg;[H254*6UsD3pXQRpmmlkMTBcY6rH&sWkUxU?1ZxL1zLtNZp&ORP[Q'4Ovj(H)P4P1RPBRpO'[4TZaHi(f4an(s-uJBp .kN[g5q"Fwtg4et4SO.q2hy6Q (9wpMBtc:c&NO*cJR0&B7I:y74utgdmmIgFgMGknQ87FtkVUu5G-(5LAn
?&tUSN8P"!8"AskX"znq761"EYhwSGyq_n!--"!]AUc1G;nvpVEWk_u!!9t,zQT7MKn)J(8b;MsVhmV8x&]QJ;[Udm;(J
S5zlBzE- f?Pd ']I?ePv;6jc1]K0W9s:rrs aR8Q7vKxLexcDtfok_5]i"vjseP
hBI,vN-eW!C9
]r8r.A&1]TF


In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batches(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out    

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step : {iter}, train loss : {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    xb, yb = get_batches('train')

    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


step : 0, train loss : 4.344, val loss: 4.358
step : 250, train loss : 4.292, val loss: 4.285
step : 500, train loss : 4.234, val loss: 4.227
step : 750, train loss : 4.167, val loss: 4.176
4.088095188140869


In [16]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


)ocjV,*WskXuGF51qZxXRhNqjs]2hn!!!xK'WQbtNQ&oy]K!E?8rDU&3fMKfePUu
C'4(]I!uPyKTL[ToyLQd42 8pShBc1tc
BtTEQR:IZ)icX(5UW[DEJOx22AI:?c4,Xuun!xL8SM .rVVlBs SN8xY87yS[95AsemyyD1
Fw9vbEM:IdR'J(3BIGk22S]5n6 A(EP-AI!"Yv1]L
, hp"Orc7my3(HUBt5n,*Fg4UrriUUKnUs&SBayRSSzmWPF:DS5-n4I!u*Y85Vp aml6DePIXLj7mY8eP4"QWyY)C]"Yr0?8d6D,6nmJq5;
e.U(
XrizyUH(3OzO9&4u
)MGPF-hnC4)O
tIG3Ruq3pylj;sa?7FR]ss&1Fw4Ur0"vG4P1'bMG*bzjanvjcQCBPEEdUfAq1Y-D:IoDpVQVV*w;MQUZxi2DtxL1GIorfpn!?FX!Q1OE*4q]Mzfo6D*slyjY-hd?VVhuQg.voc4F:ig6N8rtc
